In [9]:
import keyring
from pprint import pprint

api_key_irp = keyring.get_password("irp_api_key", "birdblues")
api_secret_irp = keyring.get_password("irp_api_secret", "birdblues")

api_key_isa = keyring.get_password("isa_api_key", "birdblues")
api_secret_isa = keyring.get_password("isa_api_secret", "birdblues")

api_key_0 = keyring.get_password("api_key_0", "birdblues")
api_secret_0 = keyring.get_password("api_secret_0","birdblues")

api_key_1 = keyring.get_password("api_key_1", "birdblues")
api_secret_1 = keyring.get_password("api_secret_1","birdblues")

api_key_2 = keyring.get_password("api_key_2", "birdblues")
api_secret_2 = keyring.get_password("api_secret_2","birdblues")

# print(f"api_key_irp = {api_key_irp}")
# print(f"api_secret_irp = {api_secret_irp}")
# print(f"api_key_isa = {api_key_isa}")
# print(f"api_secret_isa = {api_secret_isa}")
# print(f"api_key_0 = {api_key_0}")
# print(f"api_secret_0 = {api_secret_0}")
# print(f"api_key_1 = {api_key_1}")
# print(f"api_secret_1 = {api_secret_1}")
# print(f"api_key_2 = {api_key_2}")
# print(f"api_secret_2 = {api_secret_2}")

In [10]:
acc_no_0 = "63981494-01" # 위탁
acc_no_1 = "64267316-22" # 연금저축 1
acc_no_2 = "64012548-22" # 연금저축 2
acc_no_isa = "64043278-01" # ISA
acc_no_irp = "64012548-29" # IR

In [11]:
import keyring

token_0 = keyring.get_password("access_token_0", "birdblues")
token_1 = keyring.get_password("access_token_1", "birdblues")
token_2 = keyring.get_password("access_token_2", "birdblues")
token_3 = keyring.get_password("access_token_3", "birdblues")
token_4 = keyring.get_password("access_token_4", "birdblues")

# print(f"token_0 = {token_0}")
# print(f"token_1 = {token_1}")
# print(f"token_2 = {token_2}")
# print(f"token_3 = {token_3}")
# print(f"token_4 = {token_4}")

In [12]:
import requests
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

column_mapping = {
    'trad_dt': '거래일',
    'pdno': '코드',
    'prdt_name': '상품명',
    'trad_dvsn_name': '거래구분',
    'loan_dt': '대출일',
    'hldg_qty': '보유',
    'pchs_unpr': '매입가격',
    'buy_qty': '매수수량',
    'buy_amt': '매수금액',
    'sll_pric': '매도가격',
    'sll_qty': '매도수량',
    'sll_amt': '매도금액',
    'rlzt_pfls': '실현손익',
    'pfls_rt': '손익률',
    'fee': '수수료',
    'tl_tax': '세금',
    'loan_int': '대출이자'
}

cols_to_convert = ['보유', '매입가격', '매수수량', '매수금액', '매도가격', '매도수량', '매도금액', '실현손익', '손익률', '수수료', '세금', '대출이자']

def get_trade_profit(access_token, app_key, app_secret, acc_no):
    
    output1_list = []
    
    url_base = "https://openapi.koreainvestment.com:9443"
    path = "/uapi/domestic-stock/v1/trading/inquire-period-trade-profit"
    url = f"{url_base}/{path}"

    headers = {
        "Content-Type": "application/json; charset=utf-8",
        "authorization": f"Bearer {access_token}",
        "appKey": app_key,
        "appSecret": app_secret,
        "tr_id": "TTTC8715R"
        # "tr_cont": "",
        # "custtype": "P"
        
    }

    params = {
        "CANO": acc_no[:8],  # 계좌번호 앞 8지리
        "ACNT_PRDT_CD": acc_no[-2:],  # 계좌번호 뒤 2자리
        "SORT_DVSN": "01",
        "PDNO": "",
        "INQR_STRT_DT": "20230101",
        "INQR_END_DT": "20241231",
        "CBLC_DVSN": "00",
        "CTX_AREA_NK100": "",
        "CTX_AREA_FK100": ""
    }
    res = requests.get(url, headers=headers, params=params)
    body = res.json()
    if 'output1' in body:
        output1_list.extend(body['output1'])
        
    # display(body)
    # print(res.headers['tr_cont'])

    while res.headers['tr_cont'] == "M":
        headers["tr_cont"] = res.headers['tr_cont']
        params["INQR_STRT_DT"] = body['ctx_area_nk100'][:8]
        params["CTX_AREA_NK100"] = body['ctx_area_nk100']
        params["CTX_AREA_FK100"] = body['ctx_area_fk100']
        res = requests.get(url, headers=headers, params=params)
        body = res.json()
        if 'output1' in body:
            output1_list.extend(body['output1'])
        # display(body)
        # print(res.headers['tr_cont'])
    df = pd.DataFrame(output1_list)
    df.rename(columns=column_mapping, inplace=True)
 
    # 각 컬럼에 대해서 쉼표 제거하고 숫자형으로 변환
    for col in cols_to_convert:
        df[col] = df[col].str.replace(',', '')  # 쉼표 제거
        df[col] = pd.to_numeric(df[col], errors='coerce')  # 숫자형으로 변환 (에러가 발생하면 NaN으로 처리)
        
    df['계좌'] = acc_no
    df['정산금'] = df['매도금액'] - df['매수금액'] - df['수수료'] - df['세금'] - df['대출이자']

    df['거래일'] = pd.to_datetime(df['거래일'], format='%Y%m%d')
    # df = df[['거래일', '종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '코드', '계좌']]
    df.set_index('거래일', inplace=True)
    df.sort_index(inplace=True)
    df.drop_duplicates(inplace=True)
    
    return df

In [13]:
# access_token_0 = get_access_token(api_key_0, api_secret_0)
# access_token_1 = get_access_token(api_key_1, api_secret_1)
# access_token_2 = get_access_token(api_key_2, api_secret_2)
# access_token_3 = get_access_token(api_key_isa, api_secret_isa)
# access_token_4 = get_access_token(api_key_irp, api_secret_irp)

df1 = get_trade_profit(token_0, api_key_0, api_secret_0, acc_no_0)
df2 = get_trade_profit(token_1, api_key_1, api_secret_1, acc_no_1)
df3 = get_trade_profit(token_2, api_key_2, api_secret_2, acc_no_2)
df4 = get_trade_profit(token_3, api_key_isa, api_secret_isa, acc_no_isa)
df5 = get_trade_profit(token_4, api_key_irp, api_secret_irp, acc_no_irp)

df = pd.concat([df1, df2, df3, df4, df5])
df.sort_index(inplace=True)
display(df)


,코드,상품명,거래구분,대출일,보유,매입가격,매수수량,매수금액,매도가격,매도수량,매도금액,실현손익,손익률,수수료,세금,대출이자,계좌,정산금
거래일,,,,,,,,,,,,,,,,,,
2023-12-22,148070,KOSEF 국고채10년,현금,,1,114685,1,114685,0,0,0,0,0.00,0,0,0,64012548-29,-114685
2023-12-22,195980,PLUS 신흥국MSCI(합성 H),현금,,6,8970,6,53820,0,0,0,0,0.00,0,0,0,64012548-29,-53820
2023-12-22,272580,TIGER 단기채권액티브,현금,,2,53095,2,106190,0,0,0,0,0.00,0,0,0,64012548-29,-106190
2023-12-22,284430,KODEX 200미국채혼합,현금,,23,12460,23,286580,0,0,0,0,0.00,0,0,0,64012548-29,-286580
2023-12-22,360750,TIGER 미국S&P500,현금,,11,15435,11,169785,0,0,0,0,0.00,0,0,0,64012548-29,-169785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09,218420,KODEX 미국S&P500에너지(합성),현금,,74,13365,74,989010,0,0,0,-41,0.00,41,0,0,64043278-01,-989051
2024-09-10,218420,KODEX 미국S&P500에너지(합성),현금,,99,13371,25,334750,0,0,0,-14,0.00,14,0,0,64043278-01,-334764
2024-09-10,396500,TIGER Fn반도체TOP10,현금,,0,12019,0,0,9965,40,398600,-82178,-17.09,16,0,0,64267316-22,398584


In [6]:
# '코드'를 기준으로 데이터를 나누고 MultiIndex 컬럼 생성
def clean_code(df):
    df = df.pivot_table(index=['거래일'], columns='코드', values=['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌'], aggfunc='first')
    df['상품명'] = df['상품명'].apply(lambda col: col.ffill().bfill())
    df['계좌'] = df['계좌'].apply(lambda col: col.ffill().bfill())
    df['종류'] = df['종류'].apply(lambda col: col.ffill().bfill())
    df.fillna(0, inplace=True)

    new_order = ['종류', '거래금', '수량', '정산금', '수수료', '세금', '상품명', '계좌']
    df = df.reorder_levels([1, 0], axis=1).sort_index(axis=1)
    df = df.reindex(columns=pd.MultiIndex.from_product([df.columns.levels[0], new_order]))
    
    return df 

# 주식 = clean_code(df)
# display(주식.columns.levels[0])

# for code in 주식.columns.levels[0]:
#     display(주식[code])

# 매수 = df[df['종류'] == '거래소매수']
# 매도 = df[df['종류'] == '거래소매도']

# display(매수)

# 결과 확인
# df_114800 = 주식.loc[:, ('114800', slice(None))]
# display(df_114800)

# dd_114800 = df[df['코드'] == '114800']
# dd_114800

In [14]:
import keyring
import pandas as pd
from sqlalchemy import create_engine

password = keyring.get_password("mysql-rocketpig", "root")
user = 'root'
host = '220.76.59.96'
port = '3306'  # 기본적으로 MySQL은 3306 포트를 사용합니다
db = 'stock_db'

# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8")

# 연결 테스트
with engine.connect() as connection:
    print("SQLAlchemy로 MySQL 연결 성공!")

SQLAlchemy로 MySQL 연결 성공!


In [16]:
df.to_sql('stock', engine, if_exists='replace')
# df_buy.to_sql('stock_buy', engine, if_exists='replace')
# df_sell.to_sql('stock_sell', engine, if_exists='replace')

602

In [17]:
query = "SELECT * FROM stock"
주식 = pd.read_sql(query, con=engine)
주식.set_index('거래일', inplace=True)
주식

,코드,상품명,거래구분,대출일,보유,매입가격,매수수량,매수금액,매도가격,매도수량,매도금액,실현손익,손익률,수수료,세금,대출이자,계좌,정산금
거래일,,,,,,,,,,,,,,,,,,
2023-12-22,148070,KOSEF 국고채10년,현금,,1,114685,1,114685,0,0,0,0,0.00,0,0,0,64012548-29,-114685
2023-12-22,195980,PLUS 신흥국MSCI(합성 H),현금,,6,8970,6,53820,0,0,0,0,0.00,0,0,0,64012548-29,-53820
2023-12-22,272580,TIGER 단기채권액티브,현금,,2,53095,2,106190,0,0,0,0,0.00,0,0,0,64012548-29,-106190
2023-12-22,284430,KODEX 200미국채혼합,현금,,23,12460,23,286580,0,0,0,0,0.00,0,0,0,64012548-29,-286580
2023-12-22,360750,TIGER 미국S&P500,현금,,11,15435,11,169785,0,0,0,0,0.00,0,0,0,64012548-29,-169785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-09,218420,KODEX 미국S&P500에너지(합성),현금,,74,13365,74,989010,0,0,0,-41,0.00,41,0,0,64043278-01,-989051
2024-09-10,218420,KODEX 미국S&P500에너지(합성),현금,,99,13371,25,334750,0,0,0,-14,0.00,14,0,0,64043278-01,-334764
2024-09-10,396500,TIGER Fn반도체TOP10,현금,,0,12019,0,0,9965,40,398600,-82178,-17.09,16,0,0,64267316-22,398584


In [10]:
import yfinance as yf

ticker = '114800.KS' 
stock_data = yf.download(ticker, start="2023-01-01")
close_prices = stock_data['Close']
close_prices = close_prices.resample('D').ffill()
print(close_prices)

[*********************100%%**********************]  1 of 1 completed

Date
2023-01-02   5225.00
2023-01-03   5225.00
2023-01-04   5115.00
2023-01-05   5085.00
2023-01-06   5030.00
               ...  
2024-09-06   4465.00
2024-09-07   4465.00
2024-09-08   4465.00
2024-09-09   4490.00
2024-09-10   4515.00
Freq: D, Name: Close, Length: 618, dtype: float64


In [8]:
df3 = get_trade_profit(token_2, api_key_2, api_secret_2, acc_no_2)
df3

{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '20240214^00000A472870^300^1^00000000^                                                               ',
 'output1': [{'trad_dt': '20231227',
   'pdno': '423160',
   'prdt_name': 'KODEX KOFR금리액티브(합성)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '50',
   'pchs_unpr': '105320',
   'buy_qty': '50',
   'buy_amt': '5266000',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-221',
   'pfls_rt': '0.00000000',
   'fee': '221',
   'tl_tax': '0',
   'loan_int': '0'},
  {'trad_dt': '20231227',
   'pdno': '472870',
   'prdt_name': 'RISE 미국30년국채엔화노출(합성 H)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '10',
   'pchs_unpr': '9895',
   'buy_qty': '10',
   'buy_amt': '98950',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-4',
   'pfls_rt': '0.00000000',
   'fee': '4',
   'tl_ta

{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '20240401^00000A396500^300^1^00000000^                                                               ',
 'output1': [{'trad_dt': '20240214',
   'pdno': '396500',
   'prdt_name': 'TIGER Fn반도체TOP10',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '10',
   'pchs_unpr': '10203',
   'buy_qty': '0',
   'buy_amt': '0',
   'sll_pric': '11115',
   'sll_qty': '10',
   'sll_amt': '111150',
   'rlzt_pfls': '9115',
   'pfls_rt': '8.93355941',
   'fee': '4',
   'tl_tax': '0',
   'loan_int': '0'},
  {'trad_dt': '20240214',
   'pdno': '472870',
   'prdt_name': 'RISE 미국30년국채엔화노출(합성 H)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '600',
   'pchs_unpr': '9343',
   'buy_qty': '100',
   'buy_amt': '896350',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-37',
   'pfls_rt': '0.00000000',
   'fee': '37',
   'tl_

{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '20240419^00000A360200^300^1^00000000^                                                               ',
 'output1': [{'trad_dt': '20240401',
   'pdno': '396500',
   'prdt_name': 'TIGER Fn반도체TOP10',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '24',
   'pchs_unpr': '11924',
   'buy_qty': '10',
   'buy_amt': '135750',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-5',
   'pfls_rt': '0.00000000',
   'fee': '5',
   'tl_tax': '0',
   'loan_int': '0'},
  {'trad_dt': '20240401',
   'pdno': '423160',
   'prdt_name': 'KODEX KOFR금리액티브(합성)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '52',
   'pchs_unpr': '106283',
   'buy_qty': '12',
   'buy_amt': '1275780',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-53',
   'pfls_rt': '0.00000000',
   'fee': '53',
   'tl_tax': '0

{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '20240626^00000A449170^300^1^00000000^                                                               ',
 'output1': [{'trad_dt': '20240419',
   'pdno': '360200',
   'prdt_name': 'ACE 미국S&P500',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '26',
   'pchs_unpr': '16886',
   'buy_qty': '10',
   'buy_amt': '174700',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-7',
   'pfls_rt': '0.00000000',
   'fee': '7',
   'tl_tax': '0',
   'loan_int': '0'},
  {'trad_dt': '20240419',
   'pdno': '396500',
   'prdt_name': 'TIGER Fn반도체TOP10',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '150',
   'pchs_unpr': '12836',
   'buy_qty': '90',
   'buy_amt': '1136300',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-47',
   'pfls_rt': '0.00000000',
   'fee': '47',
   'tl_tax': '0',
   '

{'ctx_area_fk100': '                                                                                                    ',
 'ctx_area_nk100': '                                                                                                    ',
 'output1': [{'trad_dt': '20240626',
   'pdno': '449170',
   'prdt_name': 'TIGER KOFR금리액티브(합성)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '15',
   'pchs_unpr': '105663',
   'buy_qty': '0',
   'buy_amt': '0',
   'sll_pric': '105725',
   'sll_qty': '20',
   'sll_amt': '2114500',
   'rlzt_pfls': '1135',
   'pfls_rt': '0.05370806',
   'fee': '88',
   'tl_tax': '0',
   'loan_int': '0'},
  {'trad_dt': '20240626',
   'pdno': '473330',
   'prdt_name': 'SOL 미국30년국채커버드콜(합성)',
   'trad_dvsn_name': '현금',
   'loan_dt': '',
   'hldg_qty': '199',
   'pchs_unpr': '10292',
   'buy_qty': '199',
   'buy_amt': '2048165',
   'sll_pric': '0',
   'sll_qty': '0',
   'sll_amt': '0',
   'rlzt_pfls': '-86',
   'pfls_rt': '0.00000000',
   'fee': '86',
 

,코드,상품명,거래구분,대출일,보유,매입가격,매수수량,매수금액,매도가격,매도수량,매도금액,실현손익,손익률,수수료,세금,대출이자,계좌,정산금
거래일,,,,,,,,,,,,,,,,,,
2023-12-27,423160,KODEX KOFR금리액티브(합성),현금,,50,105320,50,5266000,0,0,0,-221,0.00,221,0,0,64012548-22,-5266221
2023-12-27,472870,RISE 미국30년국채엔화노출(합성 H),현금,,10,9895,10,98950,0,0,0,-4,0.00,4,0,0,64012548-22,-98954
2023-12-28,472870,RISE 미국30년국채엔화노출(합성 H),현금,,20,9977,10,100600,0,0,0,-4,0.00,4,0,0,64012548-22,-100604
2024-01-02,472870,RISE 미국30년국채엔화노출(합성 H),현금,,30,9976,10,99750,0,0,0,-4,0.00,4,0,0,64012548-22,-99754
2024-01-03,472870,RISE 미국30년국채엔화노출(합성 H),현금,,40,9982,10,100000,0,0,0,-4,0.00,4,0,0,64012548-22,-100004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-03,473330,SOL 미국30년국채커버드콜(합성),현금,,300,10145,9,89730,0,0,0,-3,0.00,3,0,0,64012548-22,-89733
2024-09-03,475380,RISE 글로벌리얼티인컴,현금,,4,10505,4,42020,0,0,0,-1,0.00,1,0,0,64012548-22,-42021
2024-09-03,489250,KODEX 미국배당다우존스,현금,,0,10005,0,0,10325,1,10325,320,3.20,0,0,0,64012548-22,10325
